![](https://1.bp.blogspot.com/-EKYS2teelzM/WgB-t2lzzqI/AAAAAAAAAgY/iWsXcbqbHsMy9dkQQFzepN4Z2jQXEf-AQCLcBGAs/s1600/z.jpg)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train=pd.read_csv('../input/big-mart-sales-prediction/Train.csv')
test=pd.read_csv('../input/big-mart-sales-prediction/Test.csv')

# EDA

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.shape

train datset has 8523 rows and 12 columns

In [ ]:
test.shape

test dataset has 5681 rows and 11 columns

In [ ]:
#We will combine train and test data for better Analysis
train['source']='train'
test['source']='test'
test['Item_Outlet_Sales']=0.0

#Concatenating the data in df variable
df=pd.concat([train,test],sort=False,ignore_index=True)
df

Our new dataset has 14204 rows and 13 columns

In [ ]:
df.isnull().sum()

Item weight column has about 2439 missing values and outlet size column has 4016 NaN values

In [ ]:
df.dtypes

Datset has 8 rows with object type data out of which we have created one and one is an identifier column. others we will have to encode them

In [ ]:
df.skew()

There is skewness present in the item visibility which need to be handled

In [ ]:
df.nunique()

Item_Identifier, Item_Weight, Item_Visibility, Item_MRP, Item_Outlet_Sales are continuos type of data rest are categorical.

In [ ]:
df.describe()

There are null values present in item weight as count less than total rows, mean is less than median in item mrp and outlet establishment year, rest of the features have median greater than median. Variance is almost zero in item visibility column and very high in item mrp. There are some outliers present as difference between min, max and interquartile range is unequal. Minimum value of visibility is zero that can not be posiible so we will treat it as a null value

### Univariate Analysis

In [ ]:
#Lets separate categorical features first
cat=[feature for feature in df.columns if df[feature].nunique()<20 and feature!='source']
cat

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
df['Item_Fat_Content'].value_counts().plot.pie(autopct='%1.1f%%')
plt.subplot(1,2,2)
sns.countplot(df['Item_Fat_Content'])
df['Item_Fat_Content'].value_counts()

Different low fat categories need to be merged. Same must be done with regular categories. Low Fat products are much more than Regular products.

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
df['Outlet_Identifier'].value_counts().plot.pie(autopct='%1.1f%%')
plt.ylabel('')
plt.subplot(2,1,2)
sns.countplot(df['Outlet_Identifier'])
df['Outlet_Identifier'].value_counts()

There are 10 outlets which are almost balanced except for two outlets

In [ ]:
plt.figure(figsize=(10,15))
plt.subplot(2,1,1)
df['Item_Type'].value_counts().plot.pie(autopct='%1.1f%%',textprops={'fontsize':10})
plt.ylabel('')
plt.subplot(2,1,2)
sns.countplot(df['Item_Type'])
plt.xticks(rotation = 90)
df['Item_Type'].value_counts()

Fruits and Vegetables and Snack Foods are the two categories in which most of the items fall whereas Seafood is the least

In [ ]:
plt.figure(figsize=(10,15))
plt.subplot(2,1,1)
df['Outlet_Establishment_Year'].value_counts().plot.pie(autopct='%1.1f%%')
plt.subplot(2,1,2)
sns.countplot(df['Outlet_Establishment_Year'])
df['Outlet_Establishment_Year'].value_counts()

Most outlets were opened in 1985 and least in 1998

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
df['Outlet_Size'].value_counts().plot.pie(autopct='%1.1f%%')
plt.subplot(1,2,2)
sns.countplot(df['Outlet_Size'])
df['Outlet_Size'].value_counts()

There are few outlets with high size. most of the outlets are of medium size.

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
df['Outlet_Location_Type'].value_counts().plot.pie(autopct='%1.1f%%')
plt.subplot(1,2,2)
sns.countplot(df['Outlet_Location_Type'])
df['Outlet_Location_Type'].value_counts()

Most of the stores are located in tier 2 cities/

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
df['Outlet_Type'].value_counts().plot.pie(autopct='%1.1f%%')
plt.subplot(1,2,2)
sns.countplot(df['Outlet_Type'])
plt.xticks(rotation=45)
df['Outlet_Type'].value_counts()

65% of the outlets are of supermarket type 1

In [ ]:
#Separating the continuous feature
cont=[feature for feature in df.columns if df[feature].nunique()>20 and feature!='Item_Identifier' and feature!='Item_Outlet_Sales']
cont

In [ ]:
for feat in cont:
    sns.boxplot(df[feat])
    plt.figure()

Only Item Visibility has large no. of outliers

In [ ]:
for feat in cont:
    sns.distplot(df[feat])
    plt.figure()

Only Item visibility is skewed to the right while others almost follow the gaussian distribution.

### Bivariate Analysis

In [ ]:
sns.stripplot(df['Outlet_Location_Type'],df['Outlet_Type'])

Tier 3 cities have only Superkarket type 1 whereas Tier 1 cities have only supermarket 1 and grocery strores

In [ ]:
sns.stripplot(df['Outlet_Size'],df['Outlet_Type'])

Supermarket type 1 are of all sizes whereas grocery stores are only small and Supermarket type 2, Supermarket type 3 are of medium size only.

In [ ]:
sns.stripplot(df['Outlet_Size'],df['Outlet_Location_Type'])

Tier 2 citis have only small outlet size and high outlet size is only found in tier 3 cities

In [ ]:
sns.stripplot(df['Outlet_Establishment_Year'],df['Outlet_Type'])

Supermarket type 2 was build much later while grocery stores and supermarket are the oldest outlet type.

In [ ]:
sns.stripplot(df['Outlet_Establishment_Year'],df['Outlet_Location_Type'])

In Tier1 and tier2 cities outlets were established in 1985 whereas tier2 got outlets after 2000

In [ ]:
sns.stripplot(df['Outlet_Establishment_Year'],df['Outlet_Size'])

After 1990 no outlet of high size was established.

In [ ]:
plt.figure(figsize=(10,6))
sns.stripplot(df['Outlet_Identifier'],df['Outlet_Type'])

There is only one oulat identifier for supermarket 2 and 3 while most of the outlet identifiers belong to supermarket1

In [ ]:
plt.figure(figsize=(10,6))
sns.stripplot(df['Outlet_Identifier'],df['Outlet_Location_Type'])

Tier 3 has the most diff types of outlets, but they are almost balanced.

In [ ]:
sns.stripplot(df['Outlet_Size'],df['Outlet_Identifier'])

There is only Outlet13 with high outlet size while medium and small of outlets each have 3 outlet identifiers. 

In [ ]:
plt.figure(figsize=(10,6))
sns.stripplot(df['Outlet_Identifier'],df['Outlet_Establishment_Year'])

Outlet 27 and 19 are the oldest outlet identifier and outlet 18  is the newest, outlet no. does not hold any order towards year of establishment. 

In [ ]:
plt.figure(figsize=(10,6))
sns.stripplot(df['Outlet_Identifier'],df['Item_Visibility'])

Outlet 10 and 19 have give the highest visibilty to products. Other outlets provide almost equal visibility.

In [ ]:
plt.figure(figsize=(10,6))
sns.stripplot(df['Outlet_Type'],df['Item_Visibility'])

Products are most visible in grocery stores rather than any other super markets

In [ ]:
sns.stripplot(df['Outlet_Size'],df['Item_Visibility'])

Smallest Stores provide the most visibility to products

In [ ]:
fig,ax=plt.subplots(2,2,figsize=(18,10))
r=0
c=0
for i,n in enumerate(['Item_Fat_Content','Outlet_Establishment_Year','Outlet_Type','Item_Fat_Content']):
    if i%2==0 and i>0:
        r+=1
        c=0
    sns.boxenplot(x=df[n],y=df['Item_Weight'],ax=ax[r,c])
    c+=1

Data of weight is missing for Supermarket 3 and for the year 1985, Median of low fat content is lowest.

In [ ]:
for i in range(len(cont)):
    for j in range(i+1,len(cont)):
        sns.scatterplot(x=cont[i],y=cont[j],data=df)
        plt.figure()

There are less products with high visibility. There seem to exist no correlation between the above continuous features

In [ ]:
sns.countplot(df['Outlet_Location_Type'],hue=df['Outlet_Type'])

Tier 2 cities have the highest no. of supermarket 2 and donot habe any other type of outlet whereas tier 3 cities have all the types of outlets with less no. of each of them.

In [ ]:
sns.countplot(df['Outlet_Size'],hue=df['Outlet_Location_Type'])

Tier 3 cities have the highest no. of medioum size outlet and tier 1 cities have highest no. small size outlets.Tier 3 cities have the least no. of stores

In [ ]:
sns.lmplot(x='Item_Weight',y='Item_Outlet_Sales',data=df)

There is no relation between item weight and item outlet sales.

In [ ]:
sns.lmplot(x='Item_Visibility',y='Item_Outlet_Sales',data=df)

Ttem outlet sales sharply decreases for the most visible items

In [ ]:
sns.lmplot(x='Item_MRP',y='Item_Outlet_Sales',data=df)

As the mrp of an item increases item outlet sales also increases

In [ ]:
for i in cat:
    plt.figure(figsize=(15,6))
    sns.boxplot(x=df[i],y=df['Item_Outlet_Sales'])
    plt.xticks(rotation=45)
    plt.figure()

There is not much relation between fat content and item outlet sales. Seafoord, Vegetables and fruits contribute towards the item outlet sales. Medium size outlets and Supermarket 3 gives the most item outlet sales. Tier 1 cities donot produce least` item outlet sales compared to other types of cities.

### Multivariate Analysis

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot('Outlet_Location_Type','Item_Outlet_Sales',hue='Outlet_Type',data=df)

Tier 3 ;ocations have the most sales. It can also be seen that they have most type of outlets, hence more sales

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot('Outlet_Size','Item_Outlet_Sales',hue='Outlet_Type',data=df)

It can be seen that medium size outlets sell the most and also they have the most type of ooutlets hence the sales increases even more

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot('Item_Visibility','Item_MRP',hue='Item_Outlet_Sales',data=df,palette='rocket_r')

Item_Outlet_Sales increases with low item visibility and high item price.

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df.corr(),annot=True)

MRP has the strongest positive correlation with Sales, while cisibility and establishment show a little negative correlation. There is not much correlation between the independent features.

# Feature Engineering

##### Handling nan values 

###### Item_Visibility

In [ ]:
df['Item_Visibility'].value_counts()

As seen earlier visibilty has 0 values which is not possible as a product will have some visibility in marts

In [ ]:
#Replacing o visibility with nan values
df['Item_Visibility'].replace(0,np.nan,inplace=True)
df['Item_Visibility'].min()

In [ ]:
#Checking visibility in Outlet size with Outlet type
plt.figure(figsize=(15,6))
sns.boxplot(data=df, x='Outlet_Size', y='Item_Visibility', hue='Outlet_Type')
plt.xticks(rotation=45)

Here visibility contains missing values

In [ ]:
#Checking visibility in Outlet identifier with Outlet type
plt.figure(figsize=(15,6))
sns.boxplot(data=df, x='Outlet_Identifier', y='Item_Visibility', hue='Outlet_Type')
plt.xticks(rotation=45)

This also contain missing values 

In [ ]:
#Checking visibility in Item type with Outlet type
plt.figure(figsize=(15,6))
sns.boxplot(data=df, x='Item_Type', y='Item_Visibility', hue='Outlet_Type')
plt.xticks(rotation=45)

Here thee are no missing values for missing visibility hence this can be used to fill nan values in visibility column

In [ ]:
#Creating pivot table to help fill nan values of visibility from here
table = df.pivot_table(values='Item_Visibility', index='Item_Type', columns='Outlet_Type', aggfunc='mean')
table

In [ ]:
# replace the nan values
# define function that returns the mean values
def find_mean(x):
    return table.loc[x['Item_Type'], x['Outlet_Type']]

# replace missing values in visibility with mean values from above pivot table
df['Item_Visibility'].fillna(df[df['Item_Visibility'].isnull()].apply(find_mean, axis=1), inplace=True)

###### Item_Weight

In [ ]:
#we try to fill the nan values of wieht by using values from item identifier
wt_table = df.pivot_table(values='Item_Weight', index='Item_Identifier')
wt_table

In [ ]:
def find_wt(x):
    return wt_table.loc[x['Item_Identifier'],'Item_Weight']

df['Item_Weight'].fillna(df[df['Item_Weight'].isnull()].apply(find_wt, axis=1), inplace=True)

###### Outlet_Size

In [ ]:
df['Outlet_Size'].replace(np.NaN,'Unknown',inplace=True)

In [ ]:
from scipy.stats import mode

In [ ]:
size_table = df.pivot_table(values='Outlet_Size', index='Outlet_Type', aggfunc=(lambda x:mode(x).mode[0]))
size_table

Grocery stores are usually smaller than super markets so we will replace NaN values by small

In [ ]:
#Filling nan values with mode
df['Outlet_Size'].replace('Unknown','Small',inplace=True)

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df.isnull(),cmap='Greys')

No null values remain

##### Engineering

###### Item_Identifier

In [ ]:
df['Item_Identifier']=df['Item_Identifier'].apply(lambda x: x[:-2])

In [ ]:
df['Item_Identifier'].unique()

Now the item identifier column looks more meaningful so we will keep it.

###### Correcting year column by subtracting it from 2021

In [ ]:
df['Outlet_Establishment_Year']=(2021.0-df['Outlet_Establishment_Year'])
df.head()

###### Merging all the low fat categories to Low fat and regular categories to Regular

In [ ]:
df['Item_Fat_Content'].unique()

In [ ]:
df['Item_Fat_Content'].replace('reg','Regular',inplace=True)
df['Item_Fat_Content'].replace(['low fat','LF'],'Low Fat',inplace=True)
df['Item_Fat_Content'].unique()

Now we have only two categories in the fat content column

##### Encoding object type features

In [ ]:
#Using Ordinal Encoder for encoding object type values
from sklearn.preprocessing import OrdinalEncoder
e=OrdinalEncoder()

In [ ]:
obj=[feature for feature in df.columns if df[feature].dtypes=='O' and feature!='source']
obj

In [ ]:
for i in obj:
    df[i]=e.fit_transform(df[i].values.reshape(-1,1))

In [ ]:
df.dtypes

All columns are converted into float type except for source.

##### Removing outliers from Visibiltiy column

In [ ]:
df=df[np.abs(df.Item_Visibility-df.Item_Visibility.mean())<=(3*df.Item_Visibility.std())]

In [ ]:
df.shape

In [ ]:
sns.boxplot(df['Item_Visibility'])

##### Removing skewness from visibility column

In [ ]:
df['Item_Visibility'].skew()

In [ ]:
df['Item_Visibility']=np.sqrt(df['Item_Visibility'])

In [ ]:
df['Item_Visibility'].skew()

In [ ]:
sns.distplot(df['Item_Visibility'])

Outliers and skewness both are removed from the visibility column

##### Scaling the data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
m=MinMaxScaler()

In [ ]:
df.iloc[:,:-2]=m.fit_transform(df.iloc[:,:-2])

##### Separating the data into train and test

In [ ]:
train = df.loc[df['source']=='train']
test = df.loc[df['source']=='test']

In [ ]:
train.drop('source',axis=1,inplace=True)
test.drop(['source','Item_Outlet_Sales'],axis=1,inplace=True)

In [ ]:
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)

In [ ]:
train

In [ ]:
test

# Modelling Phase

Importing neccessary modules

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score

In [ ]:
#importing models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor

In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [ ]:
x=train.iloc[:,:-1]
y=train.iloc[:,-1]

In [ ]:
#Choosing the best random state using Logistic regression
def randomstate(a,b):
    maxx=0
    for state in range(1,201):
        xtrain,xtest,ytrain,ytest=train_test_split(a,b,test_size=0.2,random_state=state)
        model=LinearRegression()
        model.fit(xtrain,ytrain)
        p=model.predict(xtest)
        r2=r2_score(p,ytest)
        if maxx<r2:
            maxx=r2
            j=state
    return j

In [ ]:
#Creating list of models and another list mapped to their names
models=[KNeighborsRegressor(),LinearRegression(),Lasso(),Ridge(),ElasticNet(),DecisionTreeRegressor(),
       RandomForestRegressor(),AdaBoostRegressor(),GradientBoostingRegressor(),XGBRegressor()]

names=['KNeighborsRegressor','LinearRegression','Lasso','Ridge','ElasticNet','DecisionTreeRegressor',
       'RandomForestRegressor','AdaBoostRegressor','GradientBoostingRegressor','XGBRegressor']

In [ ]:
def performance(p,ytest,s,n):
    print(m)
    print('Mean Absolute Error is',np.round(mean_absolute_error(p,ytest),4))
    print('Mean Squared Error is',np.round(mean_squared_error(p,ytest),4))
    print('Root Mean Squared Error is',np.round(np.sqrt(mean_squared_error(p,ytest)),4))
    print('R2 Score is',np.round(r2_score(p,ytest),4)*100)
    print('Mean of cross validaton Score is',np.round(np.mean(s),4))
    print('--------------------------------------------------------------------------')

In [ ]:
def createmodels(model_list,independent,dependent,n):
    xtrain,xtest,ytrain,ytest=train_test_split(independent,dependent,test_size=0.2,random_state=randomstate(x,y))
    name=[]
    meanabs=[]
    meansqd=[]
    rootmeansqd=[]
    r2=[]
    mcv=[]
    
    #Creating models
    for i,model in enumerate(model_list):
        model.fit(xtrain,ytrain)
        p=model.predict(xtest)
        score=cross_val_score(model,independent,dependent,cv=10)
        
        #Calculating scores of the model and appending them to a list
        name.append(n[i])
        meanabs.append(np.round(mean_absolute_error(p,ytest),4))
        meansqd.append(np.round(mean_squared_error(p,ytest),4))
        rootmeansqd.append(np.round(np.sqrt(mean_squared_error(p,ytest)),4))
        r2.append(np.round(r2_score(p,ytest),4)*100)
        mcv.append(np.round(np.mean(score),4)*100)
    
    #Creating Dataframe
    data=pd.DataFrame()
    data['Model']=name
    data['Mean Absolute Error']=meanabs
    data['Mean Squared Error']=meansqd
    data['Root Mean Squared Error']=rootmeansqd
    data['R2 Score']=r2
    data['Mean of Cross validaton Score']=mcv
    data.set_index('Model',inplace = True)
    return data
        

In [ ]:
createmodels(models,x,y,names)

From above analysis only Random Forest, Gradient Boost and xgboost perform well with r2 scor more than 51 and mean absolute error less than 810.Though the results are not as good therefore we further try to increase the scores by Feature Selection

# Feature selection

##### Using ANOVA test

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [ ]:
selection = SelectKBest(score_func=f_classif)
fit = selection.fit(x,y)

In [ ]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Features','Score']  #naming the dataframe columns
featureScores

In [ ]:
featureScores.sort_values(by=['Score'],ascending=False)

Above are the best features with there score in decreasing order after going through anova test

##### Using feature importances of Extra trees regressor

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(x,y)

In [ ]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=x.columns)
feat_importances.nlargest(11).plot(kind='barh')
plt.show()

MRP, Outlet weight are the most important features and the list follows

In [ ]:
fi=list(feat_importances.nlargest(11).index)
fi

Above fi list contains features in order with most important feature on the top and least important feature below.

##### Using Lasso Coeficients

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid={'alpha':[1e-15,1e-10,1e-8,1e-5,1e-3,0.1,1,5,10,15,20,30,35,45,50,55,65,100,110,150,1000]}
m1=GridSearchCV(Lasso(),param_grid,scoring='neg_mean_squared_error',cv=10)
m1.fit(x,y)
print(m1.best_params_)

In [ ]:
m1=Lasso(alpha=1)
m1.fit(x,y)


In [ ]:
importance = np.abs(m1.coef_)

In [ ]:
dfcolumns = pd.DataFrame(x.columns)
dfimp=pd.DataFrame(importance)
featureScores = pd.concat([dfcolumns,dfimp],axis=1)
featureScores.columns = ['Features','Coefficients']  #naming the dataframe columns
featureScores

In [ ]:
featureScores.sort_values(by=['Coefficients'],ascending=False)

Above dataframe shows features with their coefficients values. Item Type has coefficient 0 signifying that it is least important feature according to Lasso

##### Conclusion

Anova test and feature importances tell us that Outlet_Location_Type is the least significant feature.

Feature sekection tells us that Item_Type is the least significant feature

In [ ]:
x1=x.drop('Outlet_Location_Type',axis=1)
x2=x.drop('Item_Type',axis=1)
x3=x.drop(['Outlet_Location_Type','Item_Type'],axis=1)

In [ ]:
#Testing x1 dataset
createmodels(models,x1,y,names)

In [ ]:
#Testing x2 dataset
createmodels(models,x2,y,names)

In [ ]:
#Testing x3 dataset
createmodels(models,x3,y,names)

From above analysis we see that previous models which were performimng well are the one performing well even this time and the dataset providing least Root errors and highest mean cross validation score is Dataset x2 which we get after removing Item_Type which we got using feature selection of Lasso, even though results do not vary much even after doing feature selection so we will keep the original dataset for training purposes without loosing any data.

# Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=randomstate(x2,y))

##### Random Forest

In [ ]:
params={'n_estimators':[100, 300, 500],
        'min_samples_split':[1,2,3,4],
        'min_samples_leaf':[1,2,3,4],
            'max_depth':[None,1,2,3,4,5,6,7,8,9,10]}


In [ ]:
g=GridSearchCV(RandomForestRegressor(),params,cv=5)

In [ ]:
g.fit(xtrain,ytrain)

In [ ]:
print(g.best_estimator_)
print(g.best_params_)
print(g.best_score_)

In [ ]:
m=RandomForestRegressor(max_depth=6, min_samples_leaf=4, min_samples_split=3)
m.fit(xtrain,ytrain)
p=m.predict(xtest)

In [ ]:
score=cross_val_score(m,x,y,cv=10)

In [ ]:
print('Mean Absolute Error is',np.round(mean_absolute_error(p,ytest),4))
print('Mean Squared Error is',np.round(mean_squared_error(p,ytest),4))
print('Root Mean Squared Error is',np.round(np.sqrt(mean_squared_error(p,ytest)),4))
print('R2 Score is',np.round(r2_score(p,ytest),4)*100)
print('Mean of cross validaton Score is',np.round(np.mean(score)*100,4))

##### GradientBoost

In [ ]:
params={'n_estimators':[100,200,300,400,500],
      'learning_rate':[0.05, 0.10],
      'subsample':[0.5,1],
      'max_depth':[1,2,3,4,5,6,7,8,9,10]}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
g=RandomizedSearchCV(GradientBoostingRegressor(),params,cv=5)

In [ ]:
g.fit(xtrain,ytrain)

In [ ]:
print(g.best_estimator_)
print(g.best_params_)
print(g.best_score_)

In [ ]:
m=GradientBoostingRegressor(learning_rate=0.05, subsample=1,max_depth= 2,n_estimators=200 )
m.fit(xtrain,ytrain)
p=m.predict(xtest)

In [ ]:
score=cross_val_score(m,x,y,cv=10)

In [ ]:
print('Mean Absolute Error is',np.round(mean_absolute_error(p,ytest),4))
print('Mean Squared Error is',np.round(mean_squared_error(p,ytest),4))
print('Root Mean Squared Error is',np.round(np.sqrt(mean_squared_error(p,ytest)),4))
print('R2 Score is',np.round(r2_score(p,ytest),4)*100)
print('Mean of cross validaton Score is',np.round(np.mean(score)*100,4))

##### Xtreme Gradient Boost

In [ ]:
params={
 "learning_rate"    : [0.001,0.05, 0.10, ] ,
 "max_depth"        : [ 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}

In [ ]:
g=RandomizedSearchCV(XGBRegressor(),params,cv=5)

In [ ]:
g.fit(xtrain,ytrain)

In [ ]:
print(g.best_estimator_)
print(g.best_params_)
print(g.best_score_)

In [ ]:
m=XGBRegressor(colsample_bytree= 0.7, gamma= 0.1, learning_rate=0.05, max_depth=5, min_child_weight= 3)
m.fit(xtrain,ytrain)
p=m.predict(xtest)

In [ ]:
score=cross_val_score(m,x,y,cv=10)

In [ ]:
print('Mean Absolute Error is',np.round(mean_absolute_error(p,ytest),4))
print('Mean Squared Error is',np.round(mean_squared_error(p,ytest),4))
print('Root Mean Squared Error is',np.round(np.sqrt(mean_squared_error(p,ytest)),4))
print('R2 Score is',np.round(r2_score(p,ytest),4)*100)
print('Mean of cross validaton Score is',np.round(np.mean(score)*100,4))

##### Conclusion

After Hyperparametertunning the best model with least error and highest r2 score and cross validation score is Gradient Boost

## Finalizing the best model

In [ ]:
model=GradientBoostingRegressor(learning_rate=0.05, subsample=1,max_depth= 3,n_estimators=100 )
model.fit(xtrain,ytrain)
p=model.predict(xtest)
score=cross_val_score(m,x,y,cv=10)

## Evaluation Metrics

In [ ]:
print('Mean Absolute Error is',np.round(mean_absolute_error(p,ytest),4))
print('Mean Squared Error is',np.round(mean_squared_error(p,ytest),4))
print('Root Mean Squared Error is',np.round(np.sqrt(mean_squared_error(p,ytest)),4))
print('R2 Score is',np.round(r2_score(p,ytest),4)*100)
print('Mean of cross validaton Score is',np.round(np.mean(score)*100,4))

In [ ]:
plt.scatter(x=ytest,y=p,color='r')
plt.plot(ytest,ytest,color='b')
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title('Gradient Boost Regressor')

There are still a lot of outliers in our output

## Saving the model

In [ ]:
import joblib
joblib.dump(model,'Mart_Sales.obj')

## Predicting the test dataset

In [ ]:
predictions=model.predict(test)

In [ ]:
predictions

#### Saving the predictions

In [ ]:
predictions=pd.DataFrame(predictions)

In [ ]:
predictions.to_csv('test_predictions.csv')